In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import re
import string

In [2]:
df_True=pd.read_csv('True.csv')
df_Fake=pd.read_csv('Fake.csv')

In [3]:
df_True.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
df_Fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
df_True['class']=1
df_Fake['class']=0

In [6]:
df_True.shape,df_Fake.shape

((21417, 5), (23481, 5))

In [11]:
df_merge=pd.concat([df_True,df_Fake],axis=0)

In [12]:
df_merge = df_merge.reset_index(drop=True)

In [13]:
df_merge

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [14]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   class    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [15]:
df_merge.isnull().sum()

title      0
text       0
subject    0
date       0
class      0
dtype: int64

In [16]:
df_merge.duplicated().sum()

209

In [17]:
df_merge.drop_duplicates(keep='first',inplace=True)

In [18]:
df_merge.duplicated().sum()

0

In [19]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [20]:
df_merge.drop(columns=['title','subject','date'],axis=1,inplace=True)

In [21]:
df_merge

,text,class
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1
...,...,...
44893,21st Century Wire says As 21WIRE reported earl...,0
44894,21st Century Wire says It s a familiar theme. ...,0
44895,Patrick Henningsen 21st Century WireRemember ...,0
44896,21st Century Wire says Al Jazeera America will...,0


In [22]:
shuffled_data=df_merge.sample(frac=1)

In [23]:
shuffled_data.head()

,text,class
17790,NAIROBI (Reuters) - Kenyan police fired tearga...,1
44199,Jay Dyer 21st Century WireIn the famous treati...,0
40430,CNN commentator Van Jones stood by his praise ...,0
1322,WASHINGTON (Reuters) - NFL team owners will co...,1
16077,LONDON (Reuters) - Controversies over fake new...,1


In [24]:
def wordopt(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\W', ' ', text)  # Replace non-word characters with spaces
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\n', ' ', text)  # Replace newlines with spaces
    # Consider adding steps for contractions, stop words, stemming/lemmatization
    return text

In [25]:
shuffled_data['text']=shuffled_data['text'].apply(wordopt)

In [26]:
x=shuffled_data['text']
y=shuffled_data['class']

In [27]:
x

21881    legendary journalist dan rather says that robe...
12379    abuja  reuters    a bill proposed by nigerian ...
41024                                                     
27693    the u s  has one of the highest incarceration ...
36176    no wonder we ve had the worst democrat preside...
                               ...                        
2608     washington  reuters    russia s ambassador to ...
25087    several mosques in california have received th...
24840    donald trump isn t exactly a stickler for fact...
13249    jerusalem  reuters    israeli prime minister b...
10240    washington  reuters    the white house said on...
Name: text, Length: 44689, dtype: object

In [28]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [29]:
x_train.shape,x_test.shape

((31282,), (13407,))

In [30]:
#Feature engineering: Creates numerical representations of text data that can be used by machine learning algorithms.
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization=TfidfVectorizer()
xv_train=vectorization.fit_transform(x_train)
xv_test=vectorization.transform(x_test)

In [31]:
xv_train.shape,xv_test.shape,y_train.shape

((31282, 105084), (13407, 105084), (31282,))

In [32]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(xv_train,y_train)

LogisticRegression()

In [33]:
y_pred=lr.predict(xv_test)

In [34]:
lr.score(xv_test,y_test)#accuracy score

0.986793015967766

In [35]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7059
           1       0.98      0.99      0.99      6343

    accuracy                           0.99     13402
   macro avg       0.99      0.99      0.99     13402
weighted avg       0.99      0.99      0.99     13402



In [34]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(xv_train,y_train)
y_pred=dt.predict(xv_test)

In [37]:
dt.score(xv_test,y_test)

0.995672287718251

In [38]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7059
           1       1.00      1.00      1.00      6343

    accuracy                           1.00     13402
   macro avg       1.00      1.00      1.00     13402
weighted avg       1.00      1.00      1.00     13402



In [39]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
gb.fit(xv_train,y_train)
y_pred=gb.predict(xv_test)

In [ ]:
gb.score(xv_test,y_test)

0.994776898970303

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      7059
           1       0.99      1.00      0.99      6343

    accuracy                           0.99     13402
   macro avg       0.99      0.99      0.99     13402
weighted avg       0.99      0.99      0.99     13402



In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(xv_train,y_train)
y_pred=rf.predict(xv_test)

In [ ]:
rf.score(xv_test,y_test)

0.9882107148186838

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7059
           1       0.99      0.99      0.99      6343

    accuracy                           0.99     13402
   macro avg       0.99      0.99      0.99     13402
weighted avg       0.99      0.99      0.99     13402



## Selecting 2 best models and making a pickle file

In [ ]:
import pickle
pickle.dump(dt,open('DecisionTreeClassifier.pkl','wb'))
pickle.dump(gb,open('GradientBoostingClassifier.pkl','wb'))
pickle.dump(vectorization,open('vectorization.pkl','wb'))

In [ ]:
def output_label(n):
    if n==0:
        return "Fake News"
    elif n==1:
        return "Not a fake News"
def manual_testing(news):
    testing_news={"text":[news]}
    new_def_test=pd.DataFrame(testing_news)
    new_def_test["text"]=new_def_test["text"].apply(wordopt)
    new_x_test=new_def_test["text"]
    new_xv_test=vectorization.transform(new_x_test)
    pred_lr=lr.predict(new_xv_test)
    pred_dt=dt.predict(new_xv_test)
    pred_gb=gb.predict(new_xv_test)
    pred_rf=rf.predict(new_xv_test)
    return print("\n\n LR Prediction:{} \n DT Prediction:{} \n GBC Prediction:{} \n RFC Prediction:{}".format(output_label(pred_lr[0]),
                                                                                                             output_label(pred_dt[0]),
                                                                                                             output_label(pred_gb[0]),
                                                                                                             output_label(pred_rf[0])))

In [ ]:
news=str(input())
manual_testing(news)

 Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev



 LR Prediction:Fake News 
 DT Prediction:Fake News 
 GBC Prediction:Fake News 
 RFC Prediction:Fake News
